In [1]:
# Importing the necessary packages

import numpy as np
import os, sys
import matplotlib.pyplot as plt

# importing XSPEC

import xspec as xs
xs.Xset.allowPrompting = False #keeps pyxspec from hanging, waiting a response to a prompt
xs.Xset.allowNewAttributes = True

In [2]:
N = np.linspace(1.24937577e-02, 2.76388943e-02, 30) # normalization values, [1e-52 cm^-2]

In [3]:
E_LLE = np.load('bn081024891_EMeV.npy') # LLE energy bins, [MeV]
SED18 = np.load('bn081024891_SED18.npy') # the output of Manuel's code, N = 1 cm^-2

from scipy.interpolate import interp1d
f = interp1d(E_LLE*1000, SED18/1000, 'cubic', fill_value="extrapolate") # in keV and per keV


# defining Xspec model

def ALP(engs, params, flux):
    energy_binsizes = np.ediff1d(engs)
    # finding the average bin value for energy
    energy = (np.asarray(engs[:-1])+np.asarray(engs[1:]))/2
    flux[:] = f(energy) * energy_binsizes

ALPInfo = ()
xs.AllModels.addPyMod(ALP, ALPInfo, 'add', spectrumDependent=False)

In [4]:
#bn081024891# Faking the data

xs.AllData.clear()
xs.AllModels.clear() 
m = xs.Model('ALP') 

# freezing the model parameters 
for i in range(len(N)):
    m.ALP.norm.values = N[i]
    m.ALP.norm.frozen = True
    xs.AllData.clear()
    fs = xs.FakeitSettings(background="bn081024891_LAT-LLE_bkgspectra.bak{1}",response='bn081024891_LAT-LLE_weightedrsp.rsp', fileName='bn081024891_fakeit_%s.fak' %i)
    sl = 2000*[fs] # making 2000 simulations
    xs.AllData.fakeit(2000, sl)